In [ ]:
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt

from db_utils import get_rect_to_rect_data, DatabaseConnection

In [ ]:
sumo_sim_file = "./sim_data/oranienstr_new_params.csv"

In [ ]:
start_rect_coords = (13.426,52.4991, 13.4264,52.4998)
end_rect_coords = (13.4112,52.5031, 13.4117,52.5039)

In [ ]:
with DatabaseConnection() as cur:
    cur.execute("""
SELECT filename,
       ST_AsGeoJSON(geom) :: json -> 'coordinates' AS coordinates,
       timestamps,
       velos
FROM ride
WHERE st_intersects(geom,
                    st_setsrid( st_makebox2d( st_makepoint(13.4112,52.5031), st_makepoint(13.4117,52.5039)), 4326))
                AND st_intersects(geom,
                    st_setsrid( st_makebox2d( st_makepoint(13.426,52.4991), st_makepoint(13.4264,52.4998)), 4326))""")
    res = cur.fetchall()
    df = pd.DataFrame(res, columns=['filename', 'coords', 'timestamps', 'velos'])

In [ ]:
df_simra_konst = df[['filename', 'timestamps', 'coords']]
# df_simra_konst.explode(['timestamps', 'coords'], ignore_index=True)

In [ ]:
df_simra = get_rect_to_rect_data(start_rect_coords, end_rect_coords)
df_simra_paths = df_simra[['filename', 'ts', 'lon', 'lat']]
df_simra_paths.rename({'filename': 'ride_id'}, axis='columns', inplace=True)

In [ ]:
df_sumo = pd.read_csv(sumo_sim_file, delimiter=';')
df_sumo_paths = df_sumo[['vehicle_id', 'timestep_time', 'vehicle_x', 'vehicle_y']]
df_sumo_paths.rename({'vehicle_id': 'ride_id', 'timestep_time': 'ts', 'vehicle_x': 'lon', 'vehicle_y': 'lat'}, axis='columns', inplace=True)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
from typing import Dict

def plot_ride_paths(ride_data: Dict[str, pd.DataFrame]):
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.set_aspect(1.5)
    
    colors = ['blue', 'orange', 'green', 'pink']

    # plot rides for each dataframe
    for data_idx, (data_name, df) in enumerate(ride_data.items()):
        df_grouped = df.groupby('ride_id')
        for ride_group_name in df_grouped.groups:
            df_ride_group = df_grouped.get_group(ride_group_name)
            ax.plot(df_ride_group.lon, df_ride_group.lat, color=colors[data_idx], label=data_name, linewidth=1)
            # add labels to legend 

    x_min = min(min(df.lon) for df in ride_data.values())
    x_max = max(max(df.lon) for df in ride_data.values())
    y_min = min(min(df.lat) for df in ride_data.values())
    y_max = max(max(df.lat) for df in ride_data.values())
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    ax.xaxis.set_major_locator(ticker.LinearLocator(4))
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
    ax.yaxis.set_major_locator(ticker.LinearLocator(4))
    ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
    ax.set_xlabel('Longitude in decimal degrees')
    ax.set_ylabel('Latitude in decimal degrees')

    # plt.savefig('simra_vs_sumo_ride_path.png', transparent=True)
    plt.legend()
    plt.show()

In [ ]:
plot_ride_paths({'SUMO': df_sumo_paths, 'SimRa': df_simra_paths})